In [1]:
import pandas as pd

In [3]:
excel_pertanyaan = pd.read_excel("pertanyaan benchmark_langchain-multi_bge-reranker-v2-m3.xlsx", sheet_name=None, index_col=0)

In [10]:
for num, pertanyaan in enumerate(excel_pertanyaan['PNPK']["Pertanyaan"]):
    pertanyaan = pertanyaan
    jawaban1 = excel_pertanyaan["PNPK"]["Dokumen 1"]
    jawaban2 = excel_pertanyaan["PNPK"]["Dokumen 2"]
    jawaban3 = excel_pertanyaan["PNPK"]["Dokumen 3"]

In [4]:
PROMPT_TEMPLATE = """
#### Instruction:
Your task is to determine if the given piece of text contains the necessary information required to answer the question. If the text contains the necessary information required to answer the question, return 1. 
If the text is does not contains the necessary information required to answer, return 0.
you dont have to complete, uncomplete text

### Example:
Question: What are the benefits of a balanced diet?
---
A balanced diet provides essential nutrients that your body needs to function correctly. This includes proteins, carbohydrates, fats, vitamins, and minerals.

Regular exercise is crucial for maintaining overall health and fitness. It can help manage weight, improve cardiovascular health, and enhance mood.

Research indicates that a diet rich in fruits and vegetables can reduce the risk of
---
### Answer:
1

### Another example:
Question: How does regular exercise benefit mental health?
---
Physical activity helps to release endorphins, which are chemicals in the brain that act as natural painkillers and mood elevators.

Eating a variety of foods can ensure you get all the nutrients your body needs, which can help improve your overall health and energy levels.

Regular exercise has been shown to reduce symptoms of anxiety and 
---
###Answer:
1

### Input
Question: {question}
---
{context}
---
###Answer:
"""

In [8]:
from langchain.prompts import ChatPromptTemplate
from llm_client import AlpacaLLM
from deep_translator import GoogleTranslator

for num, pertanyaan in enumerate(excel_pertanyaan["PNPK"]["Pertanyaan"], start=1):
    question = GoogleTranslator(source='id', target='en').translate(pertanyaan)
    lst_docs = []
    lst_docs.append(GoogleTranslator(source='id', target='en').translate(excel_pertanyaan["PNPK"]["Dokumen 1"][num])) 
    lst_docs.append(GoogleTranslator(source='id', target='en').translate(excel_pertanyaan["PNPK"]["Dokumen 2"][num]))
    lst_docs.append(GoogleTranslator(source='id', target='en').translate(excel_pertanyaan["PNPK"]["Dokumen 3"][num]))
    

    ai_response = []
    for docs in lst_docs:
        context_text = docs
        prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
        prompt = prompt_template.format(context=context_text, question=pertanyaan)

        model = AlpacaLLM()
        response_text = model.invoke(prompt)
        ai_response.append(response_text)

    formatted_response = f"Pertanyaan {num}, {pertanyaan} , hasil {ai_response}"
    print(formatted_response)

Pertanyaan 1, Apa penyebab kerusakan ginjal pada anak? , hasil ['Answer:\n0', '\nAnswer:\n0', '1\n\n### Answer:\n0']
Pertanyaan 2, Apa yang harus dijelaskan saat melakukan edukasi pra-dialisis (PEP) pada pasien? , hasil ['f) Discussion on lifestyle changes needed during treatment\nHD or PD.\ng) The importance of adherence to therapy and follow-up\nregularly.\nh) Information on support services available.\n\nAnswer:\n0', '\nAnswer:\n0', '\nAnswer:\n0']
Pertanyaan 3, Apa yang harus dipersiapkan pada pasien/ keluarga pasien sebelum dilakukan prosedur HD pada anak? , hasil ['\nAnswer:\n0', '\nAnswer:\n0', '\nAnswer:\n0']


KeyboardInterrupt: 